## Import All Required Libraries

In [11]:
# Import all connection libraries

from spotipy import SpotifyClientCredentials
import spotipy
import sqlalchemy as sql
import psycopg2

# Import all data manipulation libraries

import pandas as pd
from pandarallel import pandarallel
import numpy as np
from tqdm import tqdm
pandarallel.initialize(verbose=0, nb_workers=8)

In [12]:
from functools import lru_cache

# Use the lru_cache decorator to cache the result of the function
# maxsize=1000 means the cache will store the result of up to 1000 items
@lru_cache(maxsize=1000)
def get_playlist_tracks(playlist_uri):
    """
    This function uses the Spotify API to get the tracks of a given playlist.
    The function is decorated with the lru_cache decorator to cache the result for each unique playlist_uri.
    So, if the same playlist_uri is passed to the function again, the cached result will be returned
    instead of making a new API call.
    :param playlist_uri: The Spotify URI of the playlist
    :return: A DataFrame containing the tracks of the playlist
    """
    # Use the Spotify API to get the tracks of the playlist
    tracks = pd.json_normalize(sp.playlist_tracks(playlist_uri), record_path=['items'])
    return tracks

## Load Initial Training Data

In [13]:
# establish connection to Spotify API

cid = 'e5448a8a4fdc4b5d98b44e956d50546d'
secret = '8924c0394d3f49a4a569fc03e891aa1b'
client_credentials = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials, requests_timeout=15, retries=10)

# establish connection to Postgres

host_name = 'localhost'
database_name = 'Spotify'
engine = sql.create_engine('postgresql+psycopg2://postgres:DataNerd2023!!\
@localhost/Spotify')

# load training data
data = pd.read_csv('distinct_playlists.csv')[['playlist_uri', 'playlist_name']]
data['playlist_uri'] = data['playlist_uri'].str.strip()

data2 = pd.read_sql('SELECT DISTINCT playlist_uri FROM playlist_tracks', engine)

outer = data.merge(data2, how='outer', indicator=True)
anti_join = outer[(outer._merge=='left_only')].drop('_merge', axis=1)

data = pd.DataFrame(anti_join)
data = data[0:50]
data.head()

,playlist_uri,playlist_name
0,spotify:playlist:000J1zgXTT67NmjEPF4oFa,cleaning my room and getting distracted by old...
1,spotify:playlist:003nEQSeKn7WjQvTJL82Rd,Most Popular Rap Ever
2,spotify:playlist:004J3j7FrqRytkgHYeXo7E,angry edm vibes
3,spotify:playlist:0060A8jknJTfjlm4j57ifZ,Clean Hip-Hop Mix - 2015-17
4,spotify:playlist:008G1BbvK1NQvbAV8MHvDz,Pop Hits 2000's - Throwback Vibes


## Iterate Through Training Data to Change Grain of Data

In [14]:
test_list = []
series = data['playlist_uri'].to_dict()
for playlist in tqdm(series.values()):
    try:
        tracks = get_playlist_tracks(playlist)
        if len(tracks) >= 100:
                tracks2 = pd.json_normalize(sp.playlist_tracks(playlist, offset=100), record_path=['items'])
                tracks3 = pd.json_normalize(sp.playlist_tracks(playlist, offset=200), record_path=['items'])
                tracks = pd.concat([tracks, tracks2, tracks3])
                secondary_data = pd.json_normalize(sp.playlist(playlist))
                tracks['playlist_uri'] = playlist
                tracks = tracks.merge(secondary_data, left_on='playlist_uri', right_on='uri')
                tracks = tracks.rename(columns={"name":"playlist_name", "track.name":"track_name", "track.uri":"track_uri", "track.album.name":"album_name", "track.explicit":"isExplicit", "track.album.release_date":"release_date", "track.duration_ms":"duration_ms", "track.album.uri":"album_uri"})
                tracks = tracks.drop(columns=['track.album.artists', 'track.album.available_markets', 'track.album.images', 'track.artists', 'track.available_markets', 'images', 'tracks.items'])

                test_list.append(tracks)
        else:
                secondary_data = pd.json_normalize(sp.playlist(playlist))
                tracks['playlist_uri'] = playlist
                tracks = tracks.merge(secondary_data, left_on='playlist_uri', right_on='uri')
                tracks = tracks.rename(columns={"name":"playlist_name", "track.name":"track_name", "track.uri":"track_uri", "track.album.name":"album_name", "track.explicit":"isExplicit", "track.album.release_date":"release_date", "track.duration_ms":"duration_ms", "track.album.uri":"album_uri"})
                tracks = tracks.drop(columns=['track.album.artists', 'track.album.available_markets', 'track.album.images', 'track.artists', 'track.available_markets', 'images', 'tracks.items'])
                test_list.append(tracks)
    except:
            pass
test_list = pd.concat(test_list)
test_list.shape

100%|██████████| 50/50 [00:52<00:00,  1.04s/it]


(5682, 63)

## Upload Training Data to playlist_tracks Table

In [15]:
test_list.to_sql('playlist_tracks', engine, if_exists='append')

682

## Identify All Artists in Every Song

In [16]:
df = pd.read_sql('''SELECT DISTINCT track_uri FROM playlist_tracks ORDER BY track_uri''', engine)
df = pd.Series(df['track_uri'])
base_list = []
for track in tqdm(df):
    try:
        df2 = pd.json_normalize(sp.track(track), record_path=['artists'])
        df2['track_uri'] = track
        df2 = df2[['name', 'track_uri']]
        base_list.append(df2)
    except:
        pass
df2 = pd.concat(base_list)
df2

100%|██████████| 5181/5181 [10:28<00:00,  8.25it/s]


,name,track_uri
0,Brooks & Dunn,spotify:track:001UkMQHw4zXfFNdKpwXAF
1,Luke Combs,spotify:track:001UkMQHw4zXfFNdKpwXAF
0,蒋雪儿,spotify:track:007ibBncyX6qXsy3sSCpgV
0,The Weeknd,spotify:track:00gscXjMt09aOxodmaDeR4
0,Chris Young,spotify:track:00KyYtT6NaXwbPecina5Pj
...,...,...
1,2WEI,spotify:track:7zUS6YbAYq1lT9JTxuy7cw
0,Kane Brown,spotify:track:7zVCrzzEJU7u24sbJPXA5W
1,Lauren Alaina,spotify:track:7zVCrzzEJU7u24sbJPXA5W
0,Tiësto,spotify:track:7zWJ6Yu8DlIFxEANCkmsBb


## Get Row Numbers for Pivoting

In [28]:
df2['RN'] = df2.groupby("track_uri")["name"].rank(method="first", ascending=True)
df2

KeyError: 'Column not found: name'

## Pivot DataFrame to Fit all Artists in DataFrame

In [27]:
import numpy as np
conditions = [(df2['RN'] == 1.0), (df2['RN'] == 2.0), (df2['RN'] == 3.0), (df2['RN'] == 4.0)]
choices = ['artist1', 'artist2', 'artist3', 'artist4']
df2['artists'] = np.select(conditions, choices, default='null')
df2 = df2.pivot(index='track_uri', columns='artists', values='name')
df2

KeyError: 'RN'

## Upload All Artist Data to Database

In [ ]:
df2.to_sql('artists', con=engine, if_exists='append')

## Get All Audio Features

In [30]:
df = pd.read_sql('''SELECT DISTINCT track_uri FROM playlist_tracks ORDER BY track_uri''', engine)
df = pd.Series(df['track_uri'])
base_list = []
for track in tqdm(df):
    try:
        df3 = pd.json_normalize(sp.audio_features(track))
        df3['track_uri'] = track
        df3 = df3[['track_uri', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']]
        base_list.append(df3)
    except:
        pass
df3 = pd.concat(base_list)
df3

100%|██████████| 5181/5181 [08:11<00:00, 10.53it/s]


,track_uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,spotify:track:001UkMQHw4zXfFNdKpwXAF,0.573,0.846,7,-4.866,1,0.0344,0.003720,0.000079,0.2900,0.5620,127.061,191053,4
0,spotify:track:007ibBncyX6qXsy3sSCpgV,0.774,0.963,1,-2.428,1,0.0393,0.065100,0.140000,0.2410,0.9740,132.044,340088,4
0,spotify:track:00gscXjMt09aOxodmaDeR4,0.732,0.798,11,-6.511,0,0.0956,0.026900,0.000005,0.0718,0.9110,122.008,188918,4
0,spotify:track:00KyYtT6NaXwbPecina5Pj,0.535,0.859,6,-2.922,1,0.0344,0.052800,0.000000,0.3200,0.6980,102.008,166507,4
0,spotify:track:00Mb3DuaIH1kjrwOku9CGU,0.487,0.900,0,-4.417,1,0.0482,0.000068,0.000000,0.3580,0.4840,149.937,204000,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,spotify:track:7zpZQzTwWBN0eOsmb4WPH3,0.748,0.741,6,-3.103,1,0.0531,0.004190,0.000000,0.1470,0.5370,128.077,213053,4
0,spotify:track:7zsw78LtXUD7JfEwH64HK2,0.384,0.180,0,-17.279,1,0.1010,0.770000,0.000000,0.3670,0.3680,73.603,291693,4
0,spotify:track:7zUS6YbAYq1lT9JTxuy7cw,0.195,0.443,2,-10.856,0,0.0534,0.060000,0.006340,0.0956,0.1270,182.914,208525,3
0,spotify:track:7zVCrzzEJU7u24sbJPXA5W,0.612,0.799,10,-4.603,1,0.0275,0.008980,0.000000,0.1720,0.6870,125.976,188253,4


## Upload Audio Features to Database

In [31]:
df3.to_sql('audio_features', con=engine, if_exists='append')

180

In [32]:
all_data = pd.read_sql('''SELECT DISTINCT added_at, is_local, primary_color_x, "added_by.external_urls.spotify", "added_by.href",
"added_by.id", "added_by.type", "added_by.uri", "track.album.album_type", "track.album.external_urls.spotify", 
"track.album.href", "track.album.id", album_name, release_date, "track.album.release_date_precision", 
"track.album.total_tracks", "track.album.type", album_uri, "track.disc_number", pt.duration_ms, "track.episode",
"isExplicit", "track.external_ids.isrc", "track.external_urls.spotify", "tracks.href", "track.id", "track.is_local", 
track_name, "track.popularity", "track.preview_url", "track.track", "track.track_number", "track.type", pt.track_uri, 
"video_thumbnail.url", playlist_uri, collaborative, description, href, id, playlist_name, primary_color_y,
public, pt.snapshot_id, type, uri, "external_urls.spotify", "followers.href", "followers.total", "owner.display_name",
"owner.external_urls.spotify", "owner.href", "owner.id", "owner.type", "owner.uri", "tracks.href", "tracks.limit", "tracks.next", 
"tracks.offset", "tracks.previous", "tracks.total", track, video_thumbnail, danceability, energy, key, loudness, mode,
speechiness, acousticness, instrumentalness, liveness, valence, tempo, time_signature
FROM playlist_tracks pt
JOIN audio_features af ON pt.track_uri = af.track_uri''', engine)
f = open("s3.csv", "w")
f.truncate()
f.close()

all_data.to_csv('s3.csv', mode="w+")